In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Tue Jun 3
@author: yaning
"""

import numpy as np
import pickle
# from tensorflow.keras import datasets

path = "/home/yaning/Documents/"
np.set_printoptions(threshold=np.inf)

Import the features

In [2]:
with open(path + "gabor_data_10p_8f_two.pkl", "rb") as f:
    featured_dataset = pickle.load(f)

In [3]:
featured_dataset.shape

(1, 10, 10, 8)

Generate different firing sequences for different frequencies 

In [4]:
# form an shape (5000, 3) array with 5000 time points and 3 input neurons
# every fire is 45-50 cycles, 
pointCount = 6000
start_tstep = 500
one_cycle = np.concatenate((np.ones(45, dtype=bool), np.zeros(189, dtype=bool)))
firing_duration = 45+189
# 23*(45+189) = 5382
total_time = 5382


non_fire_array = np.zeros(pointCount, dtype=bool)

In [5]:
starting_seq = []
for num_spikes in range(23):
    num_spikes += 1
    segment_length = total_time / num_spikes

    starts = []
    for i in range(num_spikes):
        # Place each firing block in the center of its segment
        center = int((i + 0.5) * segment_length)

        start = max(0, center - firing_duration // 2)

        starts.append(start + start_tstep)
    starting_seq.append(starts)

# if zero is fire once than all neurons will fire at the middle time point
starting_seq[0] = []

In [6]:
for i in starting_seq:
    print(i)

[]
[1728, 4419]
[1280, 3074, 4868]
[1055, 2401, 3746, 5092]
[921, 1997, 3074, 4150, 5226]
[831, 1728, 2625, 3522, 4419, 5316]
[767, 1536, 2305, 3074, 3842, 4611, 5380]
[719, 1392, 2064, 2737, 3410, 4083, 4755, 5428]
[682, 1280, 1878, 2476, 3074, 3672, 4270, 4868, 5466]
[652, 1190, 1728, 2266, 2804, 3343, 3881, 4419, 4957, 5495]
[627, 1116, 1606, 2095, 2584, 3074, 3563, 4052, 4541, 5031, 5520]
[607, 1055, 1504, 1952, 2401, 2849, 3298, 3746, 4195, 4643, 5092, 5540]
[590, 1004, 1418, 1832, 2246, 2660, 3074, 3488, 3902, 4316, 4730, 5144, 5558]
[575, 959, 1344, 1728, 2112, 2497, 2881, 3266, 3650, 4035, 4419, 4803, 5188, 5572]
[562, 921, 1280, 1638, 1997, 2356, 2715, 3074, 3432, 3791, 4150, 4509, 4868, 5226, 5585]
[551, 887, 1223, 1560, 1896, 2233, 2569, 2905, 3242, 3578, 3914, 4251, 4587, 4924, 5260, 5596]
[541, 857, 1174, 1491, 1807, 2124, 2440, 2757, 3074, 3390, 3707, 4023, 4340, 4656, 4973, 5290, 5606]
[532, 831, 1130, 1429, 1728, 2027, 2326, 2625, 2924, 3223, 3522, 3821, 4120, 4419, 471

In [7]:
# fire sequence by frequency from low to high
fire_seq = []
for i in starting_seq:
    fire_array = non_fire_array.copy()
    for j in i:
        fire_array[j:j+firing_duration] = one_cycle
    fire_seq.append(fire_array)    

In [8]:
fire_seq = np.array(fire_seq)

In [9]:
fire_seq.shape

(23, 6000)

In [ ]:
with open(path + "Spiking_add_files/fire_seq_in_frequency.pkl", "wb") as f:
    pickle.dump(fire_seq, f)

Use the firing sequence with different frequencies to generate firing array for the dataset  

In [10]:
# create reference array with 0.04545 as gaps from 0-0.9999
ref = np.arange(0, 1, 0.04545)

In [11]:
ref

array([0.     , 0.04545, 0.0909 , 0.13635, 0.1818 , 0.22725, 0.2727 ,
       0.31815, 0.3636 , 0.40905, 0.4545 , 0.49995, 0.5454 , 0.59085,
       0.6363 , 0.68175, 0.7272 , 0.77265, 0.8181 , 0.86355, 0.909  ,
       0.95445, 0.9999 ])

In [12]:
featured_dataset.shape

(1, 10, 10, 8)

In [13]:
s_data = featured_dataset[:100]

In [14]:
fire_data = fire_seq[np.abs(s_data[..., None] - ref).argmin(axis=-1)]

In [15]:
fire_data.shape

(1, 10, 10, 8, 6000)

In [31]:
fire_data[0][:,:,:,4500].any()

np.True_

In [16]:
with open(path + "fire_data_10p_8f_two.pkl", "wb") as f:
    pickle.dump(fire_data, f)

Make random to True, background spantanous firing

In [21]:
p = 0.01
mask = np.random.rand(*fire_data.shape) < p

In [22]:
b_fire_data = fire_data | mask

In [23]:
with open(path + "fire_data_10p_8f_non_zero_background.pkl", "wb") as f:
    pickle.dump(b_fire_data, f)